In [1]:
import numpy as np
import pandas as pd
import plotly.express as px


In [2]:
df = pd.read_csv('../attack-sample-3m.csv')

/tmp/ipykernel_610588/446883947.py:1: DtypeWarning: Columns (0,1,3,10,11,13,14,15,18,19,20,21,22,24,25,26,27,28,30,31,32,33,34,35,36,37,42,44,50,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../attack-sample-3m.csv')


In [3]:
df.columns

Index(['frame.number', 'frame.len', 'frame.time', 'frame.time_epoch',
       'frame.protocols', 'eth.src', 'eth.dst', 'eth.type', 'ip.src', 'ip.dst',
       'ip.len', 'ip.ttl', 'ip.flags', 'ip.frag_offset', 'ip.proto',
       'ip.version', 'ip.dsfield', 'ip.checksum', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.seq', 'tcp.ack', 'tcp.flags', 'tcp.flags.syn',
       'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', 'tcp.window_size',
       'tcp.checksum', 'tcp.stream', 'udp.srcport', 'udp.dstport',
       'udp.length', 'udp.checksum', 'icmp.type', 'icmp.code', 'icmp.checksum',
       'http.request.method', 'http.request.uri', 'http.request.version',
       'http.request.full_uri', 'http.response.code', 'http.user_agent',
       'http.content_length_header', 'http.content_type', 'http.cookie',
       'http.host', 'http.referer', 'http.location', 'http.authorization',
       'http.connection', 'dns.qry.name', 'dns.qry.type', 'dns.qry.class',
       'dns.flags.response', 'dns.f

I would like to focus on some features that are more relevant to anomalous activities. I will do data exploration, statistical analysis, and visualization for each of them. These features are:

1- tcp.flags  --> how can it be anomalous?
    - Unusual combinations of TCP flags (e.g., SYN and FIN together or ACK and RST together) can indicate port scanning, network mapping, or more sophisticated attacks like session hijacking.
    - SYN flooding, which sends a large number of SYN packets without completing the three-way handshake. packets with SYN flag set but no ACK flag set within a timeout period (possible SYN flooding
    - TCP reset, where a RST packet is sent pretending to be from the other end of an established connection. RST flag set but no ACK flag set (possible TCP reset)RST flag set but no matching connection state or sequence number (TCP reset prevention),

2- ip.ttl
3- frame.protocols
4- http.request.method
5- dns.flags.response and dns.flags.rcode
6- tcp.srcport and tcp.dstport
7- ip.flags and ip.frag_offset
8- tcp.window_size
9- http.request.uri and http.request.full_uri
10- alert

In [17]:
df_tcp_flags = df[['frame.time', 'tcp.flags', 'tcp.ack', 'tcp.flags.syn', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', 'alert']]

In [19]:
df_tcp_flags['tcp.flags'].value_counts()

tcp.flags
0x0018       1920164
0x0010        868400
0x0002         84177
0x0014         71710
0x0011         29190
0x0012         20159
0x0004          2434
0x0019            16
0x0000            15
0x0029             1
0x0052             1
tcp.flags          1
0x08c2             1
Name: count, dtype: int64

These values represent different combinations of TCP flags as hexadecimal values.

FIN (0x01): Finish flag, indicates the sender has finished sending data.
SYN (0x02): Synchronize flag, used to initiate a connection.
RST (0x04): Reset flag, abruptly terminates a connection.
PSH (0x08): Push flag, asks to push the buffered data to the receiving application.
ACK (0x10): Acknowledgment flag, indicates that the acknowledgment number is valid.
URG (0x20): Urgent flag, indicates that the urgent pointer field is significant.
ECE (0x40): ECN-Echo flag, used in TCP congestion control.
CWR (0x80): Congestion Window Reduced flag, used to indicate that the sender reduced its sending rate.

* 0x0018 --> Flags: ACK (0x10) + PSH (0x08) --> Indicates that the packet is pushing data and the acknowledgment number is valid. It's common in established connections during data transfer.

* 0x0010 --> Flags: ACK (0x10) --> A standard acknowledgment packet, common in the majority of TCP connections.

* 0x0002 --> Flags: SYN (0x02) --> Indicates an attempt to open a connection; it's the first step in the three-way handshake.

* 0x0014 --> Flags: ACK (0x10) + RST (0x04) --> Meaning: Used to terminate a connection and acknowledge the receipt of a previous packet. It can indicate a normal closure or an abrupt termination.

* 0x0011 --> Flags: ACK (0x10) + FIN (0x01) --> Used to gracefully close a connection, indicating no more data from the sender and acknowledgment of the previous packet.

* 0x0012 --> Flags: ACK (0x10) + SYN (0x02) --> Part of the three-way handshake in establishing a TCP connection, acknowledging a SYN request.

* 0x0004 --> Flags: RST (0x04) --> Meaning: Used to reset the connection abruptly.

*** The remaining values are less common and might represent rare or specific situations. For instance, 0x0019 combines ACK, PSH, and FIN, which is unusual. 0x0000 indicates a packet with no flags set, which is rare and could be used in a NULL scan to probe network behavior.

Below rows are candidates for anomalies:
0x0019            16
0x0000            15
0x0029             1
0x0052             1
tcp.flags          1
0x08c2             1
 ***



Hypothesys #1:
Unusual combinations of TCP flags can indicate port scanning, network mapping, or more sophisticated attacks like session hijacking.

 SYN and FIN together -->
    - first I will need to create a filtered dataset containing only the packets with the unusual flag combinations

    - Then I will count occurrences over time or group by additional features like IP addresses or ports to see if there's a pattern or trend.Look at the IP addresses and ports involved with the unusual packets to see if they're coming from or going to known entities or a wide range of targets.

    - Analyze the timing of these packets. Are they spread out, or do they occur in bursts?

    - Visualise:
        - Time Series Plot: Show the count of unusual flag combinations over time.
        - Heatmap: If analyzing specific source/destination IPs or ports, a heatmap can show where the most activity is occurring.

    - Deep dive into specific events or patterns. Case Studies: I will pick out specific instances or bursts of unusual activity and investigate them in detail. I will look at the surrounding packets and what was happening on the network at that time.Correlation with Events: See if the spikes in unusual activity correlate with known network events, outages, or previously identified incidents.

    - Findings: Document the occurrences, patterns, and any identified anomalies.Implications: Discuss what these findings might mean for network security. Are there specific actions or investigations that should follow?

    - Adjust Parameters: Based on your findings, you might want to redefine what's considered unusual or look at additional flag combinations.Iterate: Repeat the analysis with the new parameters or as new data becomes available.

 ACK and RST together --> same as above

In [22]:
df_tcp_flags.isna().sum()

frame.time            0
tcp.flags          3731
tcp.ack            3731
tcp.flags.syn      3731
tcp.flags.ack      3731
tcp.flags.fin      3731
tcp.flags.reset    3731
alert                 0
dtype: int64

In [24]:
# SYN and FIN together -->
print(df_tcp_flags['tcp.flags.syn'].value_counts())
df_tcp_flags['tcp.flags.syn'].head(10)




tcp.flags.syn
0.0              2884003
1.0               104080
0                   7927
1                    258
tcp.flags.syn          1
Name: count, dtype: int64


0    0.0
1    0.0
2    NaN
3    1.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: tcp.flags.syn, dtype: object

In [25]:
print(df_tcp_flags['tcp.flags.fin'].value_counts())
df_tcp_flags['tcp.flags.fin'].head(10)

tcp.flags.fin
0.0              2958969
1.0                29114
0                   8092
1                     93
tcp.flags.fin          1
Name: count, dtype: int64


0    0.0
1    0.0
2    NaN
3    0.0
4    0.0
5    0.0
6    0.0
7    0.0
8    0.0
9    0.0
Name: tcp.flags.fin, dtype: object

In [26]:
# I'm looking where both of these flags are present so I'll drop the NaN rows
df_tcp_syn_fin = df_tcp_flags[['frame.time','tcp.flags.syn', 'tcp.flags.fin','alert']]

df_tcp_syn_fin = df_tcp_syn_fin.dropna()


(2996269, 4)

In [37]:
df_tcp_syn_fin[(df_tcp_syn_fin['tcp.flags.syn'] == 1.0) & (df_tcp_syn_fin['tcp.flags.fin'] == 1.0)]

,frame.time,tcp.flags.syn,tcp.flags.fin,alert


There's no packet which has both TCP SYN and TCP FIN set at the same time.I will proceed to the next hypothesis.